# Pytorch Baseline - Train

**Notes**
- Do not forget to enable the GPU (TPU) for training
- You have to add `kaggle_l5kit` as utility script
- Parts of the code below is from the [official example](https://github.com/lyft/l5kit/blob/master/examples/agent_motion_prediction/agent_motion_prediction.ipynb)
- [Baseline inference notebook](https://www.kaggle.com/pestipeti/pytorch-baseline-inference)

In [34]:
import numpy as np

import os
import torch
torch.manual_seed(0)

from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet18,resnet50,resnet101
from tqdm import tqdm
from typing import Dict
from torch import functional as F

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.rasterization import build_rasterizer
from l5kit.evaluation import write_pred_csv, compute_metrics_csv, read_gt_csv, create_chopped_dataset
from l5kit.evaluation.chop_dataset import MIN_FUTURE_STEPS
from l5kit.evaluation.metrics import neg_multi_log_likelihood, time_displace
from l5kit.geometry import transform_points
from l5kit.visualization import PREDICTED_POINTS_COLOR, TARGET_POINTS_COLOR, draw_trajectory
from prettytable import PrettyTable
from pathlib import Path

In [35]:
DIR_INPUT = "/media/ubuntu/Data/project/lyft/lyft-motion-prediction-autonomous-vehicles/"

In [37]:
cfg = {
    'format_version': 4,
    'model_params': {
        'model_architecture': 'resnet18',
        
        'history_num_frames': 10,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1
    },
    
    'raster_params': {
        'raster_size': [1, 1],
        'pixel_size': [0.5, 0.5],
        'ego_center': [0.25, 0.5],
        'map_type': 'py_semantic',
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5
    },
    
    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 32,
        'shuffle': True,
        'num_workers': 4
    },
    
    'val_data_loader': {
        'key': 'scenes/validate.zarr',
        'batch_size': 32,
        'shuffle': False,
        'num_workers': 4
    },
    
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 32,
        'shuffle': False,
        'num_workers': 4
    },
    
    'train_params': {
        'checkpoint_every_n_steps': 5000,
        'max_num_steps': 300,
        'eval_every_n_steps': 500

        
    }
}

In [38]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager(None)
VALIDATION = True

## Dataset, dataloader

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [40]:
# ===== INIT DATASET
train_cfg = cfg["train_data_loader"]

# Rasterizer
rasterizer = build_rasterizer(cfg, dm)

# Train dataset/dataloader
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)
train_dataloader = DataLoader(train_dataset,
                              shuffle=train_cfg["shuffle"],
                              batch_size=train_cfg["batch_size"])
                              #num_workers=train_cfg["num_workers"])

print(train_dataset)
print(len(train_dataset))

+------------+------------+------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   16265    |  4039527   | 320124624  |      112.19     |        248.36        |        79.25         |        24.83         |        10.00        |
+------------+------------+------------+-----------------+----------------------+----------------------+----------------------+---------------------+
22496709


In [41]:
# ===== INIT  VAL DATASET
val_cfg = cfg["val_data_loader"]

# Rasterizer
rasterizer = build_rasterizer(cfg, dm)

# Train dataset/dataloader
val_zarr = ChunkedDataset(dm.require(val_cfg["key"])).open()
val_dataset = AgentDataset(cfg, val_zarr, rasterizer)
val_dataloader = DataLoader(val_dataset,
                              shuffle=val_cfg["shuffle"],
                              batch_size=val_cfg["batch_size"])
                              #num_workers=train_cfg["num_workers"])

print(val_dataset)
print(len(val_dataset))

+------------+------------+------------+-----------------+----------------------+----------------------+----------------------+---------------------+
| Num Scenes | Num Frames | Num Agents | Total Time (hr) | Avg Frames per Scene | Avg Agents per Frame | Avg Scene Time (sec) | Avg Frame frequency |
+------------+------------+------------+-----------------+----------------------+----------------------+----------------------+---------------------+
|   16220    |  4030296   | 312617887  |      111.97     |        248.48        |        77.57         |        24.85         |        10.00        |
+------------+------------+------------+-----------------+----------------------+----------------------+----------------------+---------------------+
21624612


tr_it = iter(train_dataloader)
data = next(tr_it)
history_positions = data['history_positions']
history_positions

## Model

In [46]:
class EncoderLSTM_LyftModel(nn.Module):
    
    def __init__(self, cfg):
        super(EncoderLSTM_LyftModel, self).__init__()
        
        self.input_sz  = 2
        self.hidden_sz = 1024
        self.num_layer = 1
        self.sequence_length = 11        
        
        self.Encoder_lstm = nn.LSTM(self.input_sz,self.hidden_sz,self.num_layer,batch_first=True)
       
    def forward(self,inputs):
        
        output,hidden_state = self.Encoder_lstm(inputs)
        
        return output,hidden_state
    
class DecoderLSTM_LyftModel(nn.Module):
    def __init__(self, cfg):
        super(DecoderLSTM_LyftModel, self).__init__()
        
        self.input_sz  = 40 #(2000 from fcn_en_output reshape to 50*40)
        self.hidden_sz = 2
        self.hidden_sz_en = 1024
        self.num_layer = 1
        
        self.sequence_len_en = 11
        self.sequence_len_de = 50
        
        self.interlayer1 = 2048+1024
        self.interlayer2 = 2000
        self.interlayer3 = 64
        self.interlayer4 = 2
        

        num_targets = 2 * cfg["model_params"]["future_num_frames"]
        
        self.encoderLSTM = EncoderLSTM_LyftModel (cfg)

        
        self.Decoder_lstm = nn.LSTM( self.input_sz,self.hidden_sz,self.num_layer,batch_first=True)


        
        self.fcn_en_hidden_dec_input = nn.Sequential(nn.Linear(in_features=11264, out_features=self.interlayer1),
                            nn.ReLU(inplace=True),
                            #nn.Dropout(p=0.25),
                            nn.Linear(in_features=self.interlayer1, out_features=self.interlayer2))#2000
        
                
        self.fcn_en_hidden_dec_hidden = nn.Sequential(nn.Linear(in_features=self.hidden_sz_en, out_features=self.interlayer3),
                            nn.ReLU(inplace=True),
                            #nn.Dropout(p=0.25),
                            nn.Linear(in_features=self.interlayer3, out_features=self.interlayer4))#2
        
        
        self.fcn_en_state_dec_state= nn.Sequential(nn.Linear(in_features=self.hidden_sz_en, out_features=self.interlayer3),
                            nn.ReLU(inplace=True),
                            #nn.Dropout(p=0.25),
                            nn.Linear(in_features=self.interlayer3, out_features=self.interlayer4))#2
        


    def forward(self,inputs):

        output,hidden_state = self.encoderLSTM(inputs)
        
        #calling FCN connecting encoder and decoder
        in_to_dec            = self.fcn_en_hidden_dec_input(output.reshape(inputs.shape[0],-1))
        in_to_dec_hidden     = self.fcn_en_hidden_dec_hidden(hidden_state[0].squeeze(dim=0))
        in_to_dec_cell_state = self.fcn_en_state_dec_state(hidden_state[1].squeeze(dim=0))
        
        #reshaping

        in_to_dec             =   in_to_dec.reshape(inputs.shape[0],self.sequence_len_de,-1)
        in_to_dec_hidden      = in_to_dec_hidden.unsqueeze(dim=0)
        in_to_dec_cell_state  = in_to_dec_cell_state.unsqueeze(dim=0)
        
        decoder_out,_       = self.Decoder_lstm(in_to_dec,(in_to_dec_hidden,in_to_dec_cell_state) )          
        
        return decoder_out.reshape(inputs.shape[0],self.sequence_len_de,self.hidden_sz)

In [47]:
# ==== INIT MODEL
model = DecoderLSTM_LyftModel(cfg)
model.to(device)
#optimizer = optim.SGD(model.parameters(), lr=1e-2,momentum=0.9)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=7000,gamma=0.1)
#lr_scheduler = CyclicLR(optimizer, base_lr=1e-2, max_lr=1e-1,cycle_momentum = True)
# Later we have to filter the invalid steps.
criterion = nn.MSELoss(reduction="none")

In [48]:
model

DecoderLSTM_LyftModel(
  (encoderLSTM): EncoderLSTM_LyftModel(
    (Encoder_lstm): LSTM(2, 1024, batch_first=True)
  )
  (Decoder_lstm): LSTM(40, 2, batch_first=True)
  (fcn_en_hidden_dec_input): Sequential(
    (0): Linear(in_features=11264, out_features=3072, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=3072, out_features=2000, bias=True)
  )
  (fcn_en_hidden_dec_hidden): Sequential(
    (0): Linear(in_features=1024, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=2, bias=True)
  )
  (fcn_en_state_dec_state): Sequential(
    (0): Linear(in_features=1024, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=2, bias=True)
  )
)

In [ ]:
#checkpoint = torch.load('/media/ubuntu/Data/project/lyft/l5kit-1.0.6/examples/agent_motion_prediction/model/model_state_last_40k_18_false.pth'))
#model.load_state_dict(checkpoint['model_state_dict'])
#optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
#loss=checkpoint['loss']

In [ ]:
#device

## Training

In [49]:
# ==== TRAIN LOOP
tr_it = iter(train_dataloader)

progress_bar = tqdm(range(cfg["train_params"]["max_num_steps"]))
losses_train = []
losses_mean_train = []
losses_val = []
losses_mean_val = []

for itr in progress_bar:
    try:
        data = next(tr_it)
    except StopIteration:
        tr_it = iter(train_dataloader)
        data = next(tr_it)
    model.train()
    torch.set_grad_enabled(True)

    # Forward pass
    history_positions = data['history_positions'].to(device)
    history_availabilities = data['history_availabilities'].to(device)
    target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
    targets_position = data["target_positions"].to(device)

    outputs = model(history_positions)

    loss = criterion(outputs,targets_position)
    # not all the output steps are valid, but we can filter them out from the loss using availabilities
    loss = loss * target_availabilities
    loss = loss.mean()

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    
    losses_train.append(loss.item())
    losses_mean_train.append(np.mean(losses_train))
    
    # Validation
    if VALIDATION :#& ( cfg["train_params"]["max_num_steps"] % cfg["train_params"]["eval_every_n_steps"] ==0 ):
        vl_it = iter(val_dataloader)
        with torch.no_grad():
            try:
                val_data = next(vl_it)
            except StopIteration:
                vl_it = iter(val_dataloader)
                val_data = next(vl_it)

            model.eval()
            # Forward pass
            target_availabilities_val = val_data["target_availabilities"].unsqueeze(-1).to(device)
            targets_val = val_data["target_positions"].to(device)
            history_positions_val = data['history_positions'].to(device)
            history_availabilities_val = data['history_availabilities'].to(device)

            outputs_val = model(history_positions_val)
                    
            loss_v = criterion(outputs_val,targets_val)
            # not all the output steps are valid, but we can filter them out from the loss using availabilities
            loss_v = loss_v * target_availabilities_val
            loss_v = loss_v.mean()

            losses_val.append(loss_v.item())

            losses_mean_val.append(np.mean(losses_val))


        desc = f" TrainLoss: {round(loss.item(), 4)} ValLoss: {round(loss_v.item(), 4)} TrainMeanLoss: {np.mean(losses_train)} ValMeanLoss: {np.mean(losses_val)}" 
    else:
        desc = f" TrainLoss: {round(loss.item(), 4)}"


        #if len(losses_train)>0 and loss < min(losses_train):
        #    print(f"Loss improved from {min(losses_train)} to {loss}")
    lr_scheduler.step()

    progress_bar.set_description(desc)



  0%|          | 0/300 [00:00<?, ?it/s]

 TrainLoss: 118.0443 ValLoss: 1.5416 TrainMeanLoss: 118.0443115234375 ValMeanLoss: 1.5415878295898438:   0%|          | 0/300 [00:06<?, ?it/s]

 TrainLoss: 118.0443 ValLoss: 1.5416 TrainMeanLoss: 118.0443115234375 ValMeanLoss: 1.5415878295898438:   0%|          | 1/300 [00:06<30:04,  6.04s/it]

 TrainLoss: 57.2646 ValLoss: 1.4507 TrainMeanLoss: 87.65446662902832 ValMeanLoss: 1.4961256384849548:   0%|          | 1/300 [00:09<30:04,  6.04s/it] 

 TrainLoss: 57.2646 ValLoss: 1.4507 TrainMeanLoss: 87.65446662902832 ValMeanLoss: 1.4961256384849548:   1%|          | 2/300 [00:09<26:40,  5.37s/it]

 TrainLoss: 176.3815 ValLoss: 1.4702 TrainMeanLoss: 117.23014958699544 ValMeanLoss: 1.4874857266743977:   1%|          | 2/300 [00:13<26:40,  5.37s/it]

 TrainLoss: 176.3815 ValLoss: 1.4702 TrainMeanLoss: 117.23014958699544 ValMeanLoss: 1.4874857266743977:   1%|          | 3/300 [00:13<24:31,  4.96s/it]

 TrainLoss: 53.3591 ValLoss: 1.327 TrainMeanLoss: 10

 TrainLoss: 102.0282 ValLoss: 1.8732 TrainMeanLoss: 121.92202436006986 ValMeanLoss: 1.7971401948195238:   9%|▊         | 26/300 [01:30<13:57,  3.06s/it]

 TrainLoss: 53.8365 ValLoss: 1.9158 TrainMeanLoss: 119.40033891465929 ValMeanLoss: 1.8015349485256054:   9%|▊         | 26/300 [01:33<13:57,  3.06s/it] 

 TrainLoss: 53.8365 ValLoss: 1.9158 TrainMeanLoss: 119.40033891465929 ValMeanLoss: 1.8015349485256054:   9%|▉         | 27/300 [01:33<13:54,  3.06s/it]

 TrainLoss: 181.9562 ValLoss: 1.9369 TrainMeanLoss: 121.63447652544293 ValMeanLoss: 1.8063711524009705:   9%|▉         | 27/300 [01:36<13:54,  3.06s/it]

 TrainLoss: 181.9562 ValLoss: 1.9369 TrainMeanLoss: 121.63447652544293 ValMeanLoss: 1.8063711524009705:   9%|▉         | 28/300 [01:36<13:53,  3.07s/it]

 TrainLoss: 153.5481 ValLoss: 1.9461 TrainMeanLoss: 122.73494509992928 ValMeanLoss: 1.8111898857971718:   9%|▉         | 28/300 [01:39<13:53,  3.07s/it]

 TrainLoss: 153.5481 ValLoss: 1.9461 TrainMeanLoss: 122.73494509992928 ValMea

 TrainLoss: 91.931 ValLoss: 1.411 TrainMeanLoss: 112.48644256591797 ValMeanLoss: 1.816543684555934:  17%|█▋        | 51/300 [02:41<10:27,  2.52s/it]  

 TrainLoss: 91.931 ValLoss: 1.411 TrainMeanLoss: 112.48644256591797 ValMeanLoss: 1.816543684555934:  17%|█▋        | 52/300 [02:41<10:35,  2.56s/it]

 TrainLoss: 103.8152 ValLoss: 1.9513 TrainMeanLoss: 112.32283466267135 ValMeanLoss: 1.8190859421244208:  17%|█▋        | 52/300 [02:44<10:35,  2.56s/it]

 TrainLoss: 103.8152 ValLoss: 1.9513 TrainMeanLoss: 112.32283466267135 ValMeanLoss: 1.8190859421244208:  18%|█▊        | 53/300 [02:44<10:45,  2.61s/it]

 TrainLoss: 213.2297 ValLoss: 1.7291 TrainMeanLoss: 114.19147957695856 ValMeanLoss: 1.817419981514966:  18%|█▊        | 53/300 [02:46<10:45,  2.61s/it] 

 TrainLoss: 213.2297 ValLoss: 1.7291 TrainMeanLoss: 114.19147957695856 ValMeanLoss: 1.817419981514966:  18%|█▊        | 54/300 [02:46<10:46,  2.63s/it]

 TrainLoss: 101.6565 ValLoss: 1.9719 TrainMeanLoss: 113.96357130570846 ValMeanLoss:

 TrainLoss: 117.1834 ValLoss: 1.609 TrainMeanLoss: 119.10043652026684 ValMeanLoss: 1.8440282391263294:  26%|██▌       | 77/300 [03:44<08:42,  2.35s/it]

 TrainLoss: 175.405 ValLoss: 1.8805 TrainMeanLoss: 119.8222892956856 ValMeanLoss: 1.8444960835652473:  26%|██▌       | 77/300 [03:47<08:42,  2.35s/it] 

 TrainLoss: 175.405 ValLoss: 1.8805 TrainMeanLoss: 119.8222892956856 ValMeanLoss: 1.8444960835652473:  26%|██▌       | 78/300 [03:47<08:38,  2.33s/it]

 TrainLoss: 144.4703 ValLoss: 1.7611 TrainMeanLoss: 120.13428931900218 ValMeanLoss: 1.8434409793419173:  26%|██▌       | 78/300 [03:49<08:38,  2.33s/it]

 TrainLoss: 144.4703 ValLoss: 1.7611 TrainMeanLoss: 120.13428931900218 ValMeanLoss: 1.8434409793419173:  26%|██▋       | 79/300 [03:49<08:41,  2.36s/it]

 TrainLoss: 176.5087 ValLoss: 1.6602 TrainMeanLoss: 120.83896961212159 ValMeanLoss: 1.8411509573459626:  26%|██▋       | 79/300 [03:51<08:41,  2.36s/it]

 TrainLoss: 176.5087 ValLoss: 1.6602 TrainMeanLoss: 120.83896961212159 ValMeanLo

 TrainLoss: 91.3101 ValLoss: 1.7438 TrainMeanLoss: 117.64045562558961 ValMeanLoss: 1.8257141136428685:  34%|███▍      | 102/300 [04:46<08:02,  2.44s/it]

 TrainLoss: 91.3101 ValLoss: 1.7438 TrainMeanLoss: 117.64045562558961 ValMeanLoss: 1.8257141136428685:  34%|███▍      | 103/300 [04:46<08:00,  2.44s/it]

 TrainLoss: 73.2654 ValLoss: 1.8076 TrainMeanLoss: 117.21377250781426 ValMeanLoss: 1.8255399259237142:  34%|███▍      | 103/300 [04:48<08:00,  2.44s/it]

 TrainLoss: 73.2654 ValLoss: 1.8076 TrainMeanLoss: 117.21377250781426 ValMeanLoss: 1.8255399259237142:  35%|███▍      | 104/300 [04:48<07:57,  2.44s/it]

 TrainLoss: 60.771 ValLoss: 1.9817 TrainMeanLoss: 116.6762225105649 ValMeanLoss: 1.8270272175470987:  35%|███▍      | 104/300 [04:51<07:57,  2.44s/it]  

 TrainLoss: 60.771 ValLoss: 1.9817 TrainMeanLoss: 116.6762225105649 ValMeanLoss: 1.8270272175470987:  35%|███▌      | 105/300 [04:51<07:49,  2.41s/it]

 TrainLoss: 153.9239 ValLoss: 1.8195 TrainMeanLoss: 117.02761530426314 ValMean

...

 TrainLoss: 85.0072 ValLoss: 1.6702 TrainMeanLoss: 114.45573620822843 ValMeanLoss: 1.8349669958626091:  60%|█████▉    | 179/300 [07:49<04:40,  2.32s/it]

 TrainLoss: 122.6702 ValLoss: 1.9229 TrainMeanLoss: 114.50137184460958 ValMeanLoss: 1.8354555169741313:  60%|█████▉    | 179/300 [07:52<04:40,  2.32s/it]

 TrainLoss: 122.6702 ValLoss: 1.9229 TrainMeanLoss: 114.50137184460958 ValMeanLoss: 1.8354555169741313:  60%|██████    | 180/300 [07:52<04:35,  2.30s/it]

 TrainLoss: 91.6974 ValLoss: 1.3465 TrainMeanLoss: 114.37538310836034 ValMeanLoss: 1.8327539882607222:  60%|██████    | 180/300 [07:54<04:35,  2.30s/it] 

 TrainLoss: 91.6974 ValLoss: 1.3465 TrainMeanLoss: 114.37538310836034 ValMeanLoss: 1.8327539882607222:  60%|██████    | 181/300 [07:54<04:31,  2.28s/it]

 TrainLoss: 94.1881 ValLoss: 1.7203 TrainMeanLoss: 114.26446419495802 ValMeanLoss: 1.8321362196744144:  60%|██████    | 181/300 [07:56<04:31,  2.28s/it]

 TrainLoss: 94.1881 ValLoss: 1.7203 TrainMeanLoss: 114.26446419495802 Val

 TrainLoss: 140.5688 ValLoss: 1.8315 TrainMeanLoss: 114.69033027741968 ValMeanLoss: 1.8362114493439836:  68%|██████▊   | 204/300 [08:49<03:41,  2.31s/it]

 TrainLoss: 140.5688 ValLoss: 1.8315 TrainMeanLoss: 114.69033027741968 ValMeanLoss: 1.8362114493439836:  68%|██████▊   | 205/300 [08:49<03:29,  2.20s/it]

 TrainLoss: 117.5781 ValLoss: 1.9579 TrainMeanLoss: 114.70434836508001 ValMeanLoss: 1.836802344877743:  68%|██████▊   | 205/300 [08:52<03:29,  2.20s/it] 

 TrainLoss: 117.5781 ValLoss: 1.9579 TrainMeanLoss: 114.70434836508001 ValMeanLoss: 1.836802344877743:  69%|██████▊   | 206/300 [08:52<03:30,  2.24s/it]

 TrainLoss: 103.3338 ValLoss: 2.0156 TrainMeanLoss: 114.64941803844654 ValMeanLoss: 1.837666124537371:  69%|██████▊   | 206/300 [08:54<03:30,  2.24s/it]

 TrainLoss: 103.3338 ValLoss: 2.0156 TrainMeanLoss: 114.64941803844654 ValMeanLoss: 1.837666124537371:  69%|██████▉   | 207/300 [08:54<03:33,  2.30s/it]

 TrainLoss: 89.6794 ValLoss: 1.5529 TrainMeanLoss: 114.52936997321936 Val

 TrainLoss: 104.9768 ValLoss: 1.8077 TrainMeanLoss: 113.81370203184045 ValMeanLoss: 1.8355413965556933:  77%|███████▋  | 230/300 [09:47<02:47,  2.40s/it]

 TrainLoss: 146.1116 ValLoss: 2.1347 TrainMeanLoss: 113.95351971795549 ValMeanLoss: 1.836836443318949:  77%|███████▋  | 230/300 [09:49<02:47,  2.40s/it] 

 TrainLoss: 146.1116 ValLoss: 2.1347 TrainMeanLoss: 113.95351971795549 ValMeanLoss: 1.836836443318949:  77%|███████▋  | 231/300 [09:49<02:38,  2.30s/it]

 TrainLoss: 85.9584 ValLoss: 1.9122 TrainMeanLoss: 113.83285092074296 ValMeanLoss: 1.8371613190091889:  77%|███████▋  | 231/300 [09:52<02:38,  2.30s/it]

 TrainLoss: 85.9584 ValLoss: 1.9122 TrainMeanLoss: 113.83285092074296 ValMeanLoss: 1.8371613190091889:  77%|███████▋  | 232/300 [09:52<02:34,  2.27s/it]

 TrainLoss: 153.3059 ValLoss: 2.0152 TrainMeanLoss: 114.00226331473421 ValMeanLoss: 1.8379254668567313:  77%|███████▋  | 232/300 [09:54<02:34,  2.27s/it]

 TrainLoss: 153.3059 ValLoss: 2.0152 TrainMeanLoss: 114.00226331473421 Va

 TrainLoss: 113.9862 ValLoss: 2.1616 TrainMeanLoss: 113.22406110167503 ValMeanLoss: 1.8314938298426569:  85%|████████▌ | 255/300 [10:48<01:44,  2.33s/it]

 TrainLoss: 113.9862 ValLoss: 2.1616 TrainMeanLoss: 113.22406110167503 ValMeanLoss: 1.8314938298426569:  85%|████████▌ | 256/300 [10:48<01:40,  2.29s/it]

 TrainLoss: 51.0436 ValLoss: 1.9057 TrainMeanLoss: 112.98211377511228 ValMeanLoss: 1.8317826489994034:  85%|████████▌ | 256/300 [10:50<01:40,  2.29s/it] 

 TrainLoss: 51.0436 ValLoss: 1.9057 TrainMeanLoss: 112.98211377511228 ValMeanLoss: 1.8317826489994034:  86%|████████▌ | 257/300 [10:50<01:38,  2.29s/it]

 TrainLoss: 157.7495 ValLoss: 1.7929 TrainMeanLoss: 113.15563070312027 ValMeanLoss: 1.8316319256789924:  86%|████████▌ | 257/300 [10:52<01:38,  2.29s/it]

 TrainLoss: 157.7495 ValLoss: 1.7929 TrainMeanLoss: 113.15563070312027 ValMeanLoss: 1.8316319256789924:  86%|████████▌ | 258/300 [10:52<01:35,  2.27s/it]

 TrainLoss: 103.3018 ValLoss: 1.7948 TrainMeanLoss: 113.11758486184374 

 TrainLoss: 142.657 ValLoss: 1.6194 TrainMeanLoss: 113.88254316336744 ValMeanLoss: 1.8293292772727505:  94%|█████████▎| 281/300 [11:47<00:46,  2.47s/it]

 TrainLoss: 28.5187 ValLoss: 1.5278 TrainMeanLoss: 113.57983458633964 ValMeanLoss: 1.828260119079698:  94%|█████████▎| 281/300 [11:49<00:46,  2.47s/it] 

 TrainLoss: 28.5187 ValLoss: 1.5278 TrainMeanLoss: 113.57983458633964 ValMeanLoss: 1.828260119079698:  94%|█████████▍| 282/300 [11:49<00:45,  2.50s/it]

 TrainLoss: 99.0931 ValLoss: 1.4215 TrainMeanLoss: 113.52864458198682 ValMeanLoss: 1.8268228727178943:  94%|█████████▍| 282/300 [11:52<00:45,  2.50s/it]

 TrainLoss: 99.0931 ValLoss: 1.4215 TrainMeanLoss: 113.52864458198682 ValMeanLoss: 1.8268228727178943:  94%|█████████▍| 283/300 [11:52<00:41,  2.44s/it]

 TrainLoss: 98.2466 ValLoss: 1.833 TrainMeanLoss: 113.47483463690314 ValMeanLoss: 1.8268447127980245:  94%|█████████▍| 283/300 [11:54<00:41,  2.44s/it] 

 TrainLoss: 98.2466 ValLoss: 1.833 TrainMeanLoss: 113.47483463690314 ValMeanL

In [14]:
torch.save({
            'model_state_dict' : model.state_dict(),
            'optimizer_state_dict' : optimizer.state_dict(),
            'loss' : loss },
            '/media/ubuntu/Data/project/lyft/l5kit-1.0.6/examples/agent_motion_prediction/model/model_state_last_ENDC_lstm_10k.pth')

In [15]:
# ===== INIT DATASET
test_cfg = cfg["test_data_loader"]

# Rasterizer
rasterizer = build_rasterizer(cfg, dm)

# Test dataset/dataloader
test_zarr = ChunkedDataset(dm.require(test_cfg["key"])).open()
test_mask = np.load(f"{DIR_INPUT}/scenes/mask.npz")["arr_0"]
test_dataset = AgentDataset(cfg, test_zarr, rasterizer, agents_mask=test_mask)
test_dataloader = DataLoader(test_dataset,
                             shuffle=test_cfg["shuffle"],
                             batch_size=test_cfg["batch_size"],
                             num_workers=test_cfg["num_workers"])


print(test_dataloader)

warning, you're running with a custom agents_mask


In [16]:
model.eval()

future_coords_offsets_pd = []
timestamps = []
agent_ids = []

with torch.no_grad():
    dataiter = tqdm(test_dataloader)
    
    for data in dataiter:

        history_positions = data['history_positions'].to(device)

        outputs = model(history_positions)
        
        future_coords_offsets_pd.append(outputs.cpu().numpy().copy())
        timestamps.append(data["timestamp"].numpy().copy())
        agent_ids.append(data["track_id"].numpy().copy())


100%|██████████| 2223/2223 [14:44<00:00,  2.51it/s]


In [17]:
write_pred_csv('/media/ubuntu/Data/project/lyft/l5kit-1.0.6/examples/agent_motion_prediction/submission_ENDC_lstm_10k.csv',
               timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(future_coords_offsets_pd))

write_pred_csv('/kaggle/working/submission.csv',
               timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(future_coords_offsets_pd))